In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [2]:
url_tab = 'https://www.resetera.com/threads/media-create-sales-week-53-2020-dec-28-jan-03.356956/'


In [3]:
def obtain_table(url):
    ustitles = requests.get(url)
    soup = BeautifulSoup(ustitles.text)
    aa=soup.findAll("div", {"class": "message-cell message-cell--main"})
    return aa[0].findAll("code")[-1].text

In [4]:
text_table=obtain_table(url_tab)

In [5]:
print(text_table)

+-------+------------+------------+------------+------------+------------+-------------+
|System |  This Week |  Last Week |  Last Year |     YTD    |  Last YTD  |     LTD     |
+-------+------------+------------+------------+------------+------------+-------------+
| NSW # |    312.121 |    278.173 |    284.827 |    312.121 |    284.827 |  17.652.495 |
| PS4 # |     17.876 |     12.718 |     58.244 |     17.876 |     58.244 |   9.308.766 |
| PS5 # |     10.632 |     13.188 |            |     10.632 |            |     265.782 |
| 3DS # |        714 |        658 |      4.030 |        714 |      4.030 |  24.559.622 |
| XBS # |        134 |        205 |            |        134 |            |      31.558 |
+-------+------------+------------+------------+------------+------------+-------------+
|  ALL  |    341.477 |    304.942 |    347.213 |    341.477 |    347.213 |  51.818.223 |
+-------+------------+------------+------------+------------+------------+-------------+
|  PS5  |      7.606 

In [6]:
def tab_to_df(text_tab,year,week,days):
    tt=[]
    
    # These for bucles creates a nested list with the table
    for element in text_tab.split("+\n"):
        for element2 in element.split("\n"):
            el2=[]
            for el in element2.split("|"):
                if not((el[:1]=="\n") or (el=="") or (el[0]=="+")):
                    text_el=el.replace(".","").replace("#","(All)") # Formating the strings
                    el2.append(text_el)
            if el2 != []:
                tt.append(el2)

    for j in range(0,len(tt[0])):
        # Remove extra spaces in the header
        tt[0][j]=tt[0][j].lstrip().rstrip()
   
    for i in range(1,len(tt)):
        # Remove extra spaces in the name of the system
        tt[i][0]=tt[i][0].lstrip().rstrip() 
        for j in range(1,len(tt[0])):
            # For each cell convert the sales into an integer. 
            # If the cell is empty, avoid the error and put a 0 instead
            try:
                tt[i][j]=int(tt[i][j])
            except ValueError:
                tt[i][j]=0

    df = pd.DataFrame(tt[1:],columns=tt[0])
    
    df["Year"]=year
    df["Week"]=week
    df["Days"]=days
    return df

In [7]:
# https://www.neogaf.com/threads/media-create-sales-archive-thread-2.416253/#post-147729422

In [8]:
url="https://www.resetera.com/threads/media-create-sales-archive-thread.4115/"
ustitles = requests.get(url)
soup = BeautifulSoup(ustitles.text)
links=soup.findAll("a", {"class": "link link--internal"})  #class="link link--internal" 

In [9]:
links_clean=[]
links_data=[]
df = pd.DataFrame()
error=[]
for el in links[::-1]:
    if ( el.get('href').find("new-used") < 0):
        links_clean.append(el.get('href'))
        try:
            text_tab=obtain_table(links_clean[-1])

            temp=links_clean[-1].split(".")[2].split("-")[-6:]
            links_data.append([int(temp[1]),int(temp[0]),temp[2]+" "+temp[3]+" / "+temp[4]+" "+temp[5]])
            df=df.append(tab_to_df(text_tab,int(temp[1]),int(temp[0]),temp[2]+" "+temp[3]+" / "+temp[4]+" "+temp[5]))
        except:
            error.append(links_clean[-1])

df.reset_index()



,index,System,This Week,Last Week,Last Year,YTD,Last YTD,LTD,Year,Week,Days
0,0,NSW (All),170691,167596,96458,650408,497586,17990782.0,2021,2,jan 11 / jan 17
1,1,PS5 (All),7328,11164,0,29124,0,284274.0,2021,2,jan 11 / jan 17
2,2,PS4 (All),3422,6820,6325,28118,71928,9319008.0,2021,2,jan 11 / jan 17
3,3,3DS (All),546,551,766,1811,5739,24560719.0,2021,2,jan 11 / jan 17
4,4,XBS (All),160,109,0,403,0,31827.0,2021,2,jan 11 / jan 17
...,...,...,...,...,...,...,...,...,...,...,...
2332,10,PS4,25930,19950,26719,1204727,1157092,5009293.0,2017,42,oct 16 / oct 22
2333,11,PSV,3832,4142,7514,334911,700356,5582330.0,2017,42,oct 16 / oct 22
2334,12,n-2DSLL,8272,11563,0,344753,0,344753.0,2017,42,oct 16 / oct 22
2335,13,2DS,846,1055,8728,248057,82889,545204.0,2017,42,oct 16 / oct 22


In [10]:
error

['https://www.resetera.com/threads/media-create-sales-week-32-2019-aug-05-aug-11.134991/',
 'https://www.resetera.com/threads/media-create-sales-week-19-2019-may-06-may-12.117093/',
 'https://www.resetera.com/threads/media-create-sales-week-18-2019-apr-29-may-05.116534/',
 'https://www.resetera.com/threads/media-create-sales-week-52-2017-dec-25-dec-31.14650/']

In [11]:
df.columns

Index(['System', 'This Week', 'Last Week', 'Last Year', 'YTD', 'Last YTD',
       'LTD', 'Year', 'Week', 'Days'],
      dtype='object')

In [12]:
df

,System,This Week,Last Week,Last Year,YTD,Last YTD,LTD,Year,Week,Days
0,NSW (All),170691,167596,96458,650408,497586,17990782.0,2021,2,jan 11 / jan 17
1,PS5 (All),7328,11164,0,29124,0,284274.0,2021,2,jan 11 / jan 17
2,PS4 (All),3422,6820,6325,28118,71928,9319008.0,2021,2,jan 11 / jan 17
3,3DS (All),546,551,766,1811,5739,24560719.0,2021,2,jan 11 / jan 17
4,XBS (All),160,109,0,403,0,31827.0,2021,2,jan 11 / jan 17
...,...,...,...,...,...,...,...,...,...,...
10,PS4,25930,19950,26719,1204727,1157092,5009293.0,2017,42,oct 16 / oct 22
11,PSV,3832,4142,7514,334911,700356,5582330.0,2017,42,oct 16 / oct 22
12,n-2DSLL,8272,11563,0,344753,0,344753.0,2017,42,oct 16 / oct 22
13,2DS,846,1055,8728,248057,82889,545204.0,2017,42,oct 16 / oct 22


In [13]:
df[df["System"]=="NSW"]["This Week"].max()

303504

In [14]:
df[df["This Week"] == df[df["System"]=="NSW"]["This Week"].max()]

,System,This Week,Last Week,Last Year,YTD,Last YTD,LTD,Year,Week,Days
0,NSW,303504,230259,0,3291903,0,3291903.0,2017,51,dec 18 / dec 24
